# Github clone & ngspice setup

In [1]:
# Clone https://github.com/PipoJF/universal_filter_ngspice
!git clone https://github.com/PipoJF/universal_filter_ngspice

fatal: destination path 'universal_filter_ngspice' already exists and is not an empty directory.


In [2]:
import numpy as np
import matplotlib.pyplot as plt

!sudo apt-get install ngspice libngspice0

# Create a symbolic link for libngspice.so if it doesn't exist
# This is often needed because PySpice looks for libngspice.so (without the version number)
!if [ ! -f /usr/lib/x86_64-linux-gnu/libngspice.so ]; then \
    sudo ln -s /usr/lib/x86_64-linux-gnu/libngspice.so.0 /usr/lib/x86_64-linux-gnu/libngspice.so; \
    echo "Created symlink /usr/lib/x86_64-linux-gnu/libngspice.so"; \
fi

# Set the environment variable for PySpice to find ngspice
import os
os.environ['PYSPICE_NGSPICE_PATH'] = '/usr/lib/x86_64-linux-gnu/libngspice.so.0'

!pip install PySpice

# Diagnostic: Find the actual path of libngspice.so
# Please share the output of this command after execution.
!dpkg -L ngspice
!dpkg -L libngspice0

# Further Diagnostics: Check file existence and dependencies
!ls -l /usr/lib/x86_64-linux-gnu/libngspice.so
!ldd /usr/lib/x86_64-linux-gnu/libngspice.so

import PySpice.Logging.Logging as Logging
import logging # Import the standard logging module
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
import re
import torch
import scipy.signal as signal
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import concurrent.futures
import multiprocessing
import random
import glob

# Suppress specific PySpice warnings
logger = Logging.setup_logging()
logger.setLevel(logging.ERROR) # Use logging.ERROR instead of Logging.ERROR


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libngspice0 is already the newest version (36+ds-1ubuntu0.1).
ngspice is already the newest version (36+ds-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
/.
/usr
/usr/bin
/usr/bin/ngspice
/usr/lib
/usr/lib/x86_64-linux-gnu
/usr/lib/x86_64-linux-gnu/ngspice
/usr/lib/x86_64-linux-gnu/ngspice/analog.cm
/usr/lib/x86_64-linux-gnu/ngspice/digital.cm
/usr/lib/x86_64-linux-gnu/ngspice/spice2poly.cm
/usr/lib/x86_64-linux-gnu/ngspice/table.cm
/usr/lib/x86_64-linux-gnu/ngspice/xtradev.cm
/usr/lib/x86_64-linux-gnu/ngspice/xtraevt.cm
/usr/share
/usr/share/doc
/usr/share/doc/ngspice
/usr/share/doc/ngspice/BUGS
/usr/share/doc/ngspice/FAQ.gz
/usr/share/doc/ngspice/NEWS.gz
/usr/share/doc/ngspice/README
/usr/share/doc/ngspice/README.adms
/usr/share/doc/ngspice/README.optran
/usr/share/doc/ngspice/changelog.Debian.gz
/usr/share/doc/ngspice/contrib
/usr/share/doc/ngspice/contrib/sp

## Load model arquitectures from models.py


In [3]:
from models import AnalogLSTMModel, AnalogTCNModel, AnalogRNNModel

# More setup


In [4]:
class NetlistProcessor:
    """
    Motor de análisis de Netlists para el Kaspix Omni-Pipeline.
    Encargado de parsear, validar y extraer la configuración paramétrica
    de circuitos SPICE estándar.
    """

    def __init__(self, file_path):
        self.file_path = file_path
        self.content = ""
        self.params_raw = {}      # Diccionario original (texto, ej: '10k')
        self.params_float = {}    # Diccionario numérico (float, ej: 10000.0)
        self.io_config = {        # Metadatos de entrada/salida
            "input_source": None,
            "output_node": None,
            "ground_check": False
        }
        self.is_parsed = False

        # Carga inmediata
        self._load_file()

    def _load_file(self):
        if not os.path.exists(self.file_path):
            raise FileNotFoundError(f"[Kaspix Error] No se encuentra el archivo: {self.file_path}")

        with open(self.file_path, 'r', encoding='utf-8') as f:
            self.content = f.read()

    def _spice_to_float(self, value_str):
        """
        Convierte sufijos de ingeniería SPICE a float de Python.
        Soporta: T, G, Meg, k, m, u, n, p, f
        """
        value_str = value_str.lower().strip('{} ') # Limpiar llaves y espacios

        # Mapa de multiplicadores SPICE
        suffixes = {
            't': 1e12, 'g': 1e9, 'meg': 1e6, 'k': 1e3,
            'mil': 25.4e-6, 'm': 1e-3, 'u': 1e-6,
            'n': 1e-9, 'p': 1e-12, 'f': 1e-15
        }

        # Regex para separar número y sufijo
        # Busca un número (int o float) seguido opcionalmente de letras
        match = re.match(r'^([\d\.\-\+]+)([a-z]*)$', value_str)

        if not match:
            try:
                return float(value_str) # Intento directo
            except ValueError:
                return None # No es un número parseable (ej: una fórmula compleja)

        number, suffix = match.groups()
        multiplier = 1.0

        if suffix:
            # SPICE es 'greedy' con los sufijos, 'meg' es especial
            if suffix.startswith('meg'):
                multiplier = suffixes['meg']
            elif suffix[0] in suffixes:
                multiplier = suffixes[suffix[0]]

        return float(number) * multiplier

    def analyze(self):
        """
        Ejecuta el análisis léxico del netlist.
        """
        lines = self.content.splitlines()

        # Regex Compilados
        re_param = re.compile(r'\.param\s+(\w+)\s*=\s*(\{?[\w\.\-\+]+\}?)', re.IGNORECASE)
        re_source = re.compile(r'^\s*([vV]\w+)\s+(\w+)\s+(\w+)', re.IGNORECASE)
        re_save = re.compile(r'\.(?:save|print)\s+(?:v|tran)\s*\((.+?)\)', re.IGNORECASE)

        for line in lines:
            line = line.strip()
            if not line or line.startswith('*'): continue

            # 1. Detectar Parámetros (.param)
            pm = re_param.search(line)
            if pm:
                name, val_str = pm.groups()
                # Guardar raw
                self.params_raw[name] = val_str
                # Convertir a float
                val_float = self._spice_to_float(val_str)
                if val_float is not None:
                    self.params_float[name] = val_float

            # 2. Detectar Inputs (Heurística: Nombre contiene 'in' o 'src')
            sm = re_source.match(line)
            if sm:
                s_name, n_pos, n_neg = sm.groups()
                # Priorizamos fuentes que parezcan de señal
                if 'in' in s_name.lower() or 'src' in s_name.lower() or 'sig' in s_name.lower():
                    self.io_config["input_source"] = s_name

            # 3. Detectar Ground
            if ' 0 ' in line or line.endswith(' 0'):
                self.io_config["ground_check"] = True

            # 4. Detectar Output (.save)
            svm = re_save.search(line)
            if svm:
                # Extraer contenido dentro de paréntesis
                nodes = svm.group(1).replace(')', '').replace('(', '').split()
                if nodes:
                    self.io_config["output_node"] = nodes[0] # Tomamos el primero como principal

        # Fallback para Output si no hay .save
        if not self.io_config["output_node"]:
             if re.search(r'\b(out|output|salida)\b', self.content, re.IGNORECASE):
                 self.io_config["output_node"] = "out (Inferred)"
             else:
                 self.io_config["output_node"] = "UNKNOWN (Define .save V(node))"

        self.is_parsed = True
        return self.get_summary()

    def get_summary(self):
        if not self.is_parsed: return "No analizado."
        return {
            "file_path": self.file_path,
            "valid_spice": self.io_config["ground_check"],
            "knobs_detected": list(self.params_float.keys()),
            "knobs_values": self.params_float,
            "input_source": self.io_config["input_source"],
            "output_target": self.io_config["output_node"]
        }

In [5]:
def get_spice_ground_truth(netlist_path, input_audio, knobs_dict, fs=48000, duration=None):
    """
    Ejecuta una simulación SPICE real on-the-fly para comparar con la predicción del modelo.

    Args:
        netlist_path (str): Ruta al archivo .cir.
        input_audio (np.array): Señal de audio (numpy array, 1D).
        knobs_dict (dict): Valores FÍSICOS de los componentes {'R1': 10000, 'C1': 1e-9}.
        fs (int): Frecuencia de muestreo.

    Returns:
        np.array: Señal de voltaje de salida simulada.
    """

    # 1. Analizar el Netlist para saber nodos de entrada/salida
    processor = NetlistProcessor(netlist_path)
    meta = processor.analyze()

    if not meta['valid_spice']:
        print(f"Error: Netlist {netlist_path} inválido o sin Ground (0).")
        return np.zeros_like(input_audio)

    input_source_name = meta['input_source']
    output_node = meta['output_target']

    # 2. Configurar tiempos
    if duration is None:
        duration = len(input_audio) / fs

    # 3. Construir circuito con PySpice
    try:
        parser = SpiceParser(path=netlist_path)
        circuit = parser.build_circuit()
    except Exception as e:
        print(f"Error parseando circuito: {e}")
        return np.zeros_like(input_audio)

    # 4. Inyección de Señal (Reemplazar la fuente DC/AC por la señal de audio PWL)
    # Buscamos la fuente original definida en el .cir
    actual_source_name = None
    for element in circuit.element_names:
        if element.upper() == str(input_source_name).upper():
            actual_source_name = element
            break

    if actual_source_name:
        # Obtener nodos de la fuente original y removerla
        original_source = circuit[actual_source_name]
        n_pos, n_neg = original_source.nodes[0], original_source.nodes[1]
        circuit._elements.pop(actual_source_name)

        # Crear eje de tiempo y pares (tiempo, voltaje)
        time_axis = np.linspace(0, duration, len(input_audio))
        input_data_pwl = list(zip(time_axis.astype(float), input_audio.astype(float)))

        # Inyectar PWL (PieceWise Linear source)
        circuit.PieceWiseLinearVoltageSource('Input_Audio', n_pos, n_neg, values=input_data_pwl)
    else:
        print(f"Warning: Fuente de entrada '{input_source_name}' no encontrada en el circuito.")
        return np.zeros_like(input_audio)

    # 5. Aplicar Knobs (Modificar valores de componentes)
    for name, val in knobs_dict.items():
        if name in circuit.element_names:
            obj = circuit[name]
            val_f = float(val)
            # Lógica de asignación según tipo de componente
            if hasattr(obj, 'resistance'): obj.resistance = val_f
            elif hasattr(obj, 'capacitance'): obj.capacitance = val_f
            elif hasattr(obj, 'inductance'): obj.inductance = val_f
            elif hasattr(obj, 'voltage_gain'): obj.voltage_gain = val_f
        else:
            # Fallback a parámetros .param
            circuit.parameter(name, val)

    # 6. Simular
    try:
        simulator = circuit.simulator(temperature=25, nominal_temperature=25)
        # Usamos un paso de tiempo fino para precisión de audio
        analysis = simulator.transient(step_time=1.0/fs, end_time=duration)
    except Exception as e:
        print(f"Error en simulación NGSpice: {e}")
        return np.zeros_like(input_audio)

    # 7. Extraer Salida
    output_signal = None
    target_clean = str(output_node).upper().replace('V(', '').replace(')', '')

    for node_name in analysis.nodes:
        if str(node_name).upper() == target_clean:
            output_signal = np.array(analysis.nodes[node_name])
            break

    if output_signal is None:
        # Intento de fallback: buscar 'out' o el último nodo
        print(f"Warning: Nodo {target_clean} no encontrado. Nodos disponibles: {list(analysis.nodes.keys())}")
        return np.zeros_like(input_audio)

    # Resampleo final para asegurar coincidencia exacta con input_audio (NGSpice a veces varía el paso)
    if len(output_signal) != len(input_audio):
        output_signal = np.interp(time_axis, np.array(analysis.time), output_signal)

    return output_signal

In [6]:
# ============================================================
# CELDA 2: CONFIGURACIÓN GLOBAL
# ============================================================

# Device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️  Device: {DEVICE}")

# Rutas
WEIGHTS_DIR = '/content/universal_filter_ngspice/models_weights/passive-filters'
NETLIST_DIR = '/content/universal_filter_ngspice/circuits/passive-filters'

# Parámetros de señal
FS = 48000  # Sample rate
DURATION = 0.5  # segundos

print(f"📁 Weights dir: {WEIGHTS_DIR}")
print(f"📁 Netlist dir: {NETLIST_DIR}")


# ============================================================
# CELDA 3: CARGAR NORMALIZACIÓN DEL DATASET
# ============================================================

def load_normalization_params(checkpoint_path):
    """
    Carga kmin/kmax del checkpoint de entrenamiento.
    CRÍTICO: Usar los mismos valores que durante training.
    """
    checkpoint = torch.load(checkpoint_path, map_location='cpu')

    if 'normalization' in checkpoint:
        kmin = np.array(checkpoint['normalization']['kmin'])
        kmax = np.array(checkpoint['normalization']['kmax'])
        print("✅ Normalización cargada desde checkpoint")
    else:
        # Fallback: Valores estándar del dataset Kaspix
        print("⚠️  Normalization no encontrada. Usando valores estándar Kaspix.")
        kmin = np.array([2000.0, 2e-8, 0.1, 0.0, 0.0])
        kmax = np.array([18000.0, 1.8e-7, 180000.0, 3.6e-7, 180000.0])

    print(f"   kmin: {kmin}")
    print(f"   kmax: {kmax}")

    return kmin, kmax

# Cargar normalización desde el checkpoint LSTM (todos usan la misma)
CHECKPOINT_PATH = os.path.join(WEIGHTS_DIR, 'lstm_analog_final.pt')
KMIN, KMAX = load_normalization_params(CHECKPOINT_PATH)


# ============================================================
# CELDA 4: MAPEO DE TOPOLOGÍAS
# ============================================================

TOPOLOGY_MAP = {
    # Low-Pass Filter (Topology 0)
    'Low-Pass': 0,
    'LPF': 0,
    'Sallen-Key LPF': 0,
    'lowpass': 0,

    # High-Pass Filter (Topology 1)
    'High-Pass': 1,
    'HPF': 1,
    'Sallen-Key HPF': 1,
    'highpass': 1,

    # Bandpass Filter (Topology 2)
    'Bandpass': 2,
    'BPF': 2,
    'MFB Bandpass': 2,
    'bandpass': 2,

    # Notch Filter (Topology 3)
    'Notch': 3,
    'Twin-T': 3,
    'Twin-T Notch': 3,
    'notch': 3
}

TOPOLOGY_NAMES = {
    0: 'Low-Pass (Sallen-Key)',
    1: 'High-Pass (Sallen-Key)',
    2: 'Bandpass (MFB)',
    3: 'Notch (Twin-T)'
}

print("📋 Topologías disponibles:")
for name, idx in TOPOLOGY_MAP.items():
    if idx in TOPOLOGY_NAMES:
        print(f"   '{name}' → {idx}: {TOPOLOGY_NAMES[idx]}")


# ============================================================
# CELDA 5: FUNCIONES DE PREPARACIÓN DE INPUT
# ============================================================

def parse_component_value(val_str):
    """
    Convierte valores tipo '10k', '220n' a float.

    Examples:
        '10k' → 10000.0
        '220n' → 2.2e-7
        '1meg' → 1000000.0
    """
    if isinstance(val_str, (int, float)):
        return float(val_str)

    val_str = str(val_str).lower().strip()

    # Multiplicadores SPICE estándar
    multipliers = {
        'meg': 1e6,
        'k': 1e3,
        'm': 1e-3,
        'u': 1e-6,
        'n': 1e-9,
        'p': 1e-12
    }

    for suffix, mult in multipliers.items():
        if val_str.endswith(suffix):
            try:
                num_part = val_str[:-len(suffix)]
                return float(num_part) * mult
            except ValueError:
                pass

    # Si no tiene sufijo, intentar convertir directamente
    try:
        return float(val_str)
    except ValueError:
        print(f"⚠️  No se pudo parsear '{val_str}', usando 0.0")
        return 0.0


def prepare_input_tensor(audio, topology, knobs_dict, kmin, kmax, model_type, device='cpu'):
    """
    Prepara input tensor en formato correcto para inferencia.

    Args:
        audio: numpy array (L,) - señal de entrada
        topology: str o int - nombre o ID de topología
        knobs_dict: dict - parámetros, ej: {'R': '10k', 'C': '220n'}
        kmin, kmax: numpy arrays (5,) - parámetros de normalización
        model_type: str - 'LSTM', 'TCN', o 'RNN'
        device: str - 'cpu' o 'cuda'

    Returns:
        torch.Tensor con shape correcto:
            LSTM/RNN: (1, L, 10)
            TCN: (1, 10, L)
    """
    # Normalizar audio
    audio = np.nan_to_num(audio)
    L = len(audio)

    # Parsear y normalizar knobs
    knob_values = np.zeros(5)

    # Mapeo de nombres de componentes a índices de knobs
    knob_map = {
        # Knob 0: Resistencia principal
        'R': 0, 'Rgain': 0, 'Rseries': 0,
        # Knob 1: Capacitor principal
        'C': 1, 'Ccut': 1, 'Chp': 1, 'Cseries': 1,
        # Knob 2: Resistencia secundaria
        'R2': 2, 'Rlpf': 2, 'Rshunt': 2,
        # Knob 3: Capacitor secundario
        'C2': 3, 'Clpf': 3, 'Cshunt': 3,
        # Knob 4: Carga
        'Rload': 4
    }

    for key, val in knobs_dict.items():
        if key in knob_map:
            idx = knob_map[key]
            knob_values[idx] = parse_component_value(val)

    # Normalizar usando kmin/kmax del dataset
    knobs_norm = (knob_values - kmin) / (kmax - kmin + 1e-10)
    knobs_norm = np.clip(knobs_norm, 0, 1)

    # Topology encoding
    if isinstance(topology, str):
        topo_id = TOPOLOGY_MAP.get(topology, 0)
    else:
        topo_id = int(topology)

    topo_onehot = np.zeros(4)
    if 0 <= topo_id < 4:
        topo_onehot[topo_id] = 1.0
    else:
        print(f"⚠️  Topology ID {topo_id} inválido, usando 0")
        topo_onehot[0] = 1.0

    # Construir tensor según modelo
    if model_type in ['LSTM', 'RNN']:
        # Shape: (1, L, 10) = (batch, time, features)
        audio_t = torch.tensor(audio, dtype=torch.float32).reshape(1, L, 1)
        knobs_t = torch.tensor(knobs_norm, dtype=torch.float32).reshape(1, 1, 5).repeat(1, L, 1)
        topo_t = torch.tensor(topo_onehot, dtype=torch.float32).reshape(1, 1, 4).repeat(1, L, 1)

        tensor = torch.cat([audio_t, knobs_t, topo_t], dim=2)

    elif model_type == 'TCN':
        # Shape: (1, 10, L) = (batch, channels, time)
        tensor = torch.zeros(1, 10, L, dtype=torch.float32)
        tensor[:, 0, :] = torch.tensor(audio, dtype=torch.float32)

        for i in range(5):
            tensor[:, i+1, :] = knobs_norm[i]

        for i in range(4):
            tensor[:, i+6, :] = topo_onehot[i]

    else:
        raise ValueError(f"Modelo desconocido: {model_type}")

    return tensor.to(device)


# ============================================================
# CELDA 6: GENERADOR DE SEÑALES DE PRUEBA
# ============================================================

def generate_test_signal(signal_type='chirp', fs=48000, duration=0.5):
    """
    Genera señales de prueba estándar.

    Args:
        signal_type: 'chirp', 'noise', 'impulse', 'step', 'sine'
        fs: sample rate
        duration: duración en segundos

    Returns:
        numpy array con la señal
    """
    t = np.linspace(0, duration, int(fs * duration))

    if signal_type == 'chirp':
        # Chirp logarítmico 20Hz - 20kHz (estándar audio)
        sig = signal.chirp(t, f0=20, f1=20000, t1=duration, method='logarithmic')
        return sig * 0.8

    elif signal_type == 'noise':
        # Ruido blanco
        return np.random.uniform(-0.5, 0.5, size=len(t))

    elif signal_type == 'impulse':
        # Delta de Dirac
        sig = np.zeros(len(t))
        sig[int(0.01 * fs)] = 1.0
        return sig

    elif signal_type == 'step':
        # Step function
        sig = np.zeros(len(t))
        sig[int(0.1 * fs):] = 0.5
        return sig

    elif signal_type == 'sine':
        # Tono puro a 1 kHz
        return 0.5 * np.sin(2 * np.pi * 1000 * t)

    else:
        print(f"⚠️  Tipo '{signal_type}' desconocido, usando chirp")
        return generate_test_signal('chirp', fs, duration)


# Test de generación
print("\n🎵 Probando generador de señales:")
test_sig = generate_test_signal('chirp', FS, DURATION)
print(f"   Shape: {test_sig.shape}")
print(f"   Range: [{test_sig.min():.3f}, {test_sig.max():.3f}]")


# ============================================================
# CELDA 7: FUNCIÓN DE INFERENCIA
# ============================================================

def run_inference(model, model_type, audio, topology, knobs, device='cpu'):
    """
    Ejecuta inferencia con el modelo.

    Args:
        model: torch.nn.Module
        model_type: str - 'LSTM', 'TCN', 'RNN'
        audio: numpy array - señal de entrada
        topology: str o int
        knobs: dict - parámetros del circuito
        device: str

    Returns:
        numpy array - señal de salida predicha
    """
    model.eval()

    # Preparar input tensor
    input_tensor = prepare_input_tensor(
        audio, topology, knobs, KMIN, KMAX, model_type, device
    )

    # Inferencia
    with torch.no_grad():
        output = model(input_tensor)

        # Extraer señal (manejo robusto de shapes)
        if output.dim() == 3:
            # (batch, time, 1) → (time,)
            pred_signal = output.squeeze().cpu().numpy()
        elif output.dim() == 2:
            # (batch, time) → (time,)
            pred_signal = output.squeeze().cpu().numpy()
        else:
            pred_signal = output.cpu().numpy().flatten()

    return pred_signal


# ============================================================
# CELDA 8: CARGA DE MODELOS
# ============================================================

print("\n" + "="*70)
print("CARGANDO MODELOS")
print("="*70)

models = {}

# LSTM
try:
    lstm_path = os.path.join(WEIGHTS_DIR, 'lstm_analog_final.pt')
    lstm_model = AnalogLSTMModel(
        input_size=10,
        hidden_size=128,
        output_size=1,
        num_layers=2,
        analog_levels=32,
        noise_std=0.03
    ).to(DEVICE)

    checkpoint = torch.load(lstm_path, map_location=DEVICE)
    lstm_model.load_state_dict(checkpoint.get('model_state_dict', checkpoint), strict=False)
    lstm_model.eval()

    models['LSTM'] = lstm_model
    print("✅ LSTM cargado")
except Exception as e:
    print(f"❌ Error LSTM: {e}")

# TCN
try:
    tcn_path = os.path.join(WEIGHTS_DIR, 'tcn_analog_final.pt')
    tcn_model = AnalogTCNModel(
        input_size=10,
        channels=128,
        num_layers=12,
        kernel_size=3,
        analog_levels=32,
        noise_std=0.03
    ).to(DEVICE)

    checkpoint = torch.load(tcn_path, map_location=DEVICE)
    tcn_model.load_state_dict(checkpoint.get('model_state_dict', checkpoint), strict=False)
    tcn_model.eval()

    models['TCN'] = tcn_model
    print("✅ TCN cargado")
except Exception as e:
    print(f"❌ Error TCN: {e}")

# RNN
try:
    rnn_path = os.path.join(WEIGHTS_DIR, 'rnn_analog_final.pt')
    rnn_model = AnalogRNNModel(
        input_size=10,
        hidden_size=256,
        num_layers=3,
        analog_levels=32,
        noise_std=0.03
    ).to(DEVICE)

    checkpoint = torch.load(rnn_path, map_location=DEVICE)
    rnn_model.load_state_dict(checkpoint.get('model_state_dict', checkpoint), strict=False)
    rnn_model.eval()

    models['RNN'] = rnn_model
    print("✅ RNN cargado")
except Exception as e:
    print(f"❌ Error RNN: {e}")

print("="*70)
print(f"✅ {len(models)} modelos cargados: {list(models.keys())}")
print("="*70)


# ============================================================
# CELDA 9: FUNCIÓN PRINCIPAL DE BENCHMARK
# ============================================================

def benchmark_new_config(
    models_dict,
    topology,
    knobs,
    signal_type='chirp',
    device='cpu',
    plot=True
):
    """
    Ejecuta benchmark con configuración nueva de knobs.

    Args:
        models_dict: dict - {'LSTM': model, 'TCN': model, ...}
        topology: str - 'Low-Pass', 'High-Pass', etc.
        knobs: dict - {'R': '10k', 'C': '220n', ...}
        signal_type: str - tipo de señal de prueba
        device: str
        plot: bool - si True, genera gráficas

    Returns:
        dict con resultados por modelo
    """
    print("\n" + "="*70)
    print(f"🧪 BENCHMARK: {topology}")
    print(f"   Knobs: {knobs}")
    print(f"   Signal: {signal_type}")
    print("="*70)

    # Generar señal de prueba
    input_signal = generate_test_signal(signal_type, FS, DURATION)
    print(f"\n📊 Input signal: shape={input_signal.shape}, range=[{input_signal.min():.3f}, {input_signal.max():.3f}]")

    # Ejecutar inferencia con cada modelo
    results = {}

    for model_name, model in models_dict.items():
        try:
            print(f"\n🔹 {model_name}...")

            pred_signal = run_inference(
                model, model_name, input_signal, topology, knobs, device
            )

            # Ajustar longitudes si hay mismatch
            L = min(len(input_signal), len(pred_signal))
            pred_signal = pred_signal[:L]
            input_cropped = input_signal[:L]

            results[model_name] = {
                'signal': pred_signal,
                'input': input_cropped
            }

            print(f"   ✅ Output: shape={pred_signal.shape}, range=[{pred_signal.min():.4f}, {pred_signal.max():.4f}]")

        except Exception as e:
            print(f"   ❌ Error: {e}")
            import traceback
            traceback.print_exc()

    # Visualización
    if plot and results:
        plot_results(results, topology, knobs, signal_type)

    print("\n" + "="*70)
    print("✅ Benchmark completado")
    print("="*70)

    return results


def plot_results(results, topology, knobs, signal_type):
    """Genera gráficas de resultados"""
    fig, axes = plt.subplots(2, 1, figsize=(15, 9))

    # Obtener input de cualquier resultado
    input_signal = list(results.values())[0]['input']
    time_ms = np.arange(len(input_signal)) / FS * 1000

    # Dominio del tiempo
    axes[0].plot(time_ms[:1000], input_signal[:1000],
                'gray', ls=':', lw=2, alpha=0.5, label='Input', zorder=1)

    colors = {'LSTM': 'blue', 'TCN': 'green', 'RNN': 'red'}
    for model_name, data in results.items():
        sig = data['signal']
        axes[0].plot(time_ms[:1000], sig[:1000],
                    color=colors.get(model_name, 'purple'),
                    lw=1.8, label=model_name, alpha=0.85, zorder=2)

    axes[0].set_xlabel('Time (ms)', fontsize=12)
    axes[0].set_ylabel('Amplitude (V)', fontsize=12)
    axes[0].set_title(f'Time Domain - {topology} | {knobs} | Signal: {signal_type}',
                     fontsize=13, fontweight='bold')
    axes[0].legend(loc='upper right', fontsize=11)
    axes[0].grid(alpha=0.3)

    # Dominio de la frecuencia
    f_in, Pxx_in = signal.welch(input_signal, FS, nperseg=1024)
    axes[1].semilogx(f_in, 10 * np.log10(Pxx_in + 1e-12),
                    'gray', ls=':', lw=2, alpha=0.5, label='Input', zorder=1)

    for model_name, data in results.items():
        f, Pxx = signal.welch(data['signal'], FS, nperseg=1024)
        axes[1].semilogx(f, 10 * np.log10(Pxx + 1e-12),
                        color=colors.get(model_name, 'purple'),
                        lw=1.8, label=model_name, alpha=0.85, zorder=2)

    axes[1].set_xlabel('Frequency (Hz)', fontsize=12)
    axes[1].set_ylabel('Power Spectral Density (dB/Hz)', fontsize=12)
    axes[1].set_title('Frequency Domain', fontsize=13, fontweight='bold')
    axes[1].legend(loc='upper right', fontsize=11)
    axes[1].grid(True, which='both', alpha=0.3)
    axes[1].set_xlim([20, FS/2])

    plt.tight_layout()

    # Guardar
    filename = f"benchmark_{topology.replace(' ', '_')}_{signal_type}.png"
    plt.savefig(filename, dpi=150, bbox_inches='tight')
    print(f"\n📁 Gráfica guardada: {filename}")

    plt.show()


print("\n✅ Todas las funciones cargadas. Listo para ejecutar benchmarks!")


🖥️  Device: cpu
📁 Weights dir: /content/universal_filter_ngspice/models_weights/passive-filters
📁 Netlist dir: /content/universal_filter_ngspice/circuits/passive-filters
✅ Normalización cargada desde checkpoint
   kmin: [2.00000793e+03 2.00027461e-08 1.00074574e-01 0.00000000e+00
 0.00000000e+00]
   kmax: [1.79999922e+04 1.79999319e-07 1.79999234e+05 3.59986558e-07
 1.79998250e+05]
📋 Topologías disponibles:
   'Low-Pass' → 0: Low-Pass (Sallen-Key)
   'LPF' → 0: Low-Pass (Sallen-Key)
   'Sallen-Key LPF' → 0: Low-Pass (Sallen-Key)
   'lowpass' → 0: Low-Pass (Sallen-Key)
   'High-Pass' → 1: High-Pass (Sallen-Key)
   'HPF' → 1: High-Pass (Sallen-Key)
   'Sallen-Key HPF' → 1: High-Pass (Sallen-Key)
   'highpass' → 1: High-Pass (Sallen-Key)
   'Bandpass' → 2: Bandpass (MFB)
   'BPF' → 2: Bandpass (MFB)
   'MFB Bandpass' → 2: Bandpass (MFB)
   'bandpass' → 2: Bandpass (MFB)
   'Notch' → 3: Notch (Twin-T)
   'Twin-T' → 3: Notch (Twin-T)
   'Twin-T Notch' → 3: Notch (Twin-T)
   'notch' → 3: Not

In [7]:
# ============================================================
# CELDA: FIX DEFINITIVO PARA TCN
# ============================================================

def prepare_input_tensor_v2(audio, topology, knobs_dict, kmin, kmax, model_type, device='cpu'):
    """
    Versión 2: Maneja correctamente TCN con knobs separados
    """
    audio = np.nan_to_num(audio)
    L = len(audio)

    # Parsear knobs
    knob_values = np.zeros(5)
    knob_map = {
        'R': 0, 'Rgain': 0, 'Rseries': 0,
        'C': 1, 'Ccut': 1, 'Chp': 1, 'Cseries': 1,
        'R2': 2, 'Rlpf': 2, 'Rshunt': 2,
        'C2': 3, 'Clpf': 3, 'Cshunt': 3,
        'Rload': 4
    }

    for key, val in knobs_dict.items():
        if key in knob_map:
            idx = knob_map[key]
            knob_values[idx] = parse_component_value(val)

    # Normalizar knobs
    knobs_norm = (knob_values - kmin) / (kmax - kmin + 1e-10)
    knobs_norm = np.clip(knobs_norm, 0, 1)

    # Topology one-hot
    if isinstance(topology, str):
        topo_id = TOPOLOGY_MAP.get(topology, 0)
    else:
        topo_id = int(topology)

    topo_onehot = np.zeros(4)
    if 0 <= topo_id < 4:
        topo_onehot[topo_id] = 1.0
    else:
        topo_onehot[0] = 1.0

    # ========================================
    # CONSTRUCCIÓN SEGÚN MODELO
    # ========================================

    if model_type in ['LSTM', 'RNN']:
        # LSTM/RNN: (batch, time, features)
        # Features = Audio(1) + Knobs(5) + Topology(4) = 10
        audio_t = torch.tensor(audio, dtype=torch.float32).reshape(1, L, 1)
        knobs_t = torch.tensor(knobs_norm, dtype=torch.float32).reshape(1, 1, 5).repeat(1, L, 1)
        topo_t = torch.tensor(topo_onehot, dtype=torch.float32).reshape(1, 1, 4).repeat(1, L, 1)

        tensor = torch.cat([audio_t, knobs_t, topo_t], dim=2)
        return tensor.to(device)

    elif model_type == 'TCN':
        # TCN espera:
        # - x: (batch, channels, time) donde channels=1 (solo audio)
        # - knobs: (batch, control_features) donde control_features=9

        # Audio tensor: (1, 1, L)
        audio_tensor = torch.tensor(audio, dtype=torch.float32).reshape(1, 1, L)

        # Control: Concatenar knobs(5) + topology(4) = 9 features
        control = np.concatenate([knobs_norm, topo_onehot])
        knobs_tensor = torch.tensor(control, dtype=torch.float32).reshape(1, 9)

        return {
            'x': audio_tensor.to(device),
            'knobs': knobs_tensor.to(device)
        }

    else:
        raise ValueError(f"Modelo desconocido: {model_type}")


def run_inference_v2(model, model_type, audio, topology, knobs, device='cpu'):
    """
    Versión 2: Ejecuta inferencia con formato correcto para cada modelo
    """
    model.eval()

    # Preparar input
    input_data = prepare_input_tensor_v2(
        audio, topology, knobs, KMIN, KMAX, model_type, device
    )

    # Inferencia
    with torch.no_grad():
        if model_type == 'TCN':
            # TCN: forward(x, knobs)
            output = model(input_data['x'], input_data['knobs'])
        else:
            # LSTM/RNN: forward(x)
            output = model(input_data)

        # Extraer señal
        if output.dim() == 3:
            pred_signal = output.squeeze().cpu().numpy()
        elif output.dim() == 2:
            pred_signal = output.squeeze().cpu().numpy()
        else:
            pred_signal = output.cpu().numpy().flatten()

    return pred_signal


def benchmark_new_config_v2(
    models_dict,
    topology,
    knobs,
    signal_type='chirp',
    device='cpu',
    plot=True
):
    """
    Versión 2: Benchmark con soporte correcto para TCN
    """
    print("\n" + "="*70)
    print(f"🧪 BENCHMARK: {topology}")
    print(f"   Knobs: {knobs}")
    print(f"   Signal: {signal_type}")
    print("="*70)

    # Generar señal
    input_signal = generate_test_signal(signal_type, FS, DURATION)
    print(f"\n📊 Input signal: shape={input_signal.shape}, range=[{input_signal.min():.3f}, {input_signal.max():.3f}]")

    # Ejecutar inferencia
    results = {}

    for model_name, model in models_dict.items():
        try:
            print(f"\n🔹 {model_name}...")

            pred_signal = run_inference_v2(
                model, model_name, input_signal, topology, knobs, device
            )

            # Ajustar longitudes
            L = min(len(input_signal), len(pred_signal))
            pred_signal = pred_signal[:L]
            input_cropped = input_signal[:L]

            results[model_name] = {
                'signal': pred_signal,
                'input': input_cropped
            }

            print(f"   ✅ Output: shape={pred_signal.shape}, range=[{pred_signal.min():.4f}, {pred_signal.max():.4f}]")

        except Exception as e:
            print(f"   ❌ Error: {e}")
            import traceback
            traceback.print_exc()

    # Visualización
    if plot and results:
        plot_results(results, topology, knobs, signal_type)

    print("\n" + "="*70)
    print("✅ Benchmark completado")
    print("="*70)

    return results


print("\n✅ Funciones V2 cargadas correctamente")
print("✅ TCN ahora recibe: x=(1,1,L) + knobs=(1,9)")
print("✅ LSTM/RNN reciben: x=(1,L,10)")



✅ Funciones V2 cargadas correctamente
✅ TCN ahora recibe: x=(1,1,L) + knobs=(1,9)
✅ LSTM/RNN reciben: x=(1,L,10)


In [8]:
# ============================================================
# CELDA: FIX DEFINITIVO - TCN USA MISMO INPUT QUE LSTM/RNN
# ============================================================

def prepare_input_tensor_FINAL(audio, topology, knobs_dict, kmin, kmax, model_type, device='cpu'):
    """
    VERSIÓN FINAL: Todos los modelos usan el mismo formato de entrada
    """
    audio = np.nan_to_num(audio)
    L = len(audio)

    # Parsear knobs
    knob_values = np.zeros(5)
    knob_map = {
        'R': 0, 'Rgain': 0, 'Rseries': 0,
        'C': 1, 'Ccut': 1, 'Chp': 1, 'Cseries': 1,
        'R2': 2, 'Rlpf': 2, 'Rshunt': 2,
        'C2': 3, 'Clpf': 3, 'Cshunt': 3,
        'Rload': 4
    }

    for key, val in knobs_dict.items():
        if key in knob_map:
            idx = knob_map[key]
            knob_values[idx] = parse_component_value(val)

    # Normalizar
    knobs_norm = (knob_values - kmin) / (kmax - kmin + 1e-10)
    knobs_norm = np.clip(knobs_norm, 0, 1)

    # Topology
    if isinstance(topology, str):
        topo_id = TOPOLOGY_MAP.get(topology, 0)
    else:
        topo_id = int(topology)

    topo_onehot = np.zeros(4)
    if 0 <= topo_id < 4:
        topo_onehot[topo_id] = 1.0
    else:
        topo_onehot[0] = 1.0

    # ========================================
    # TODOS LOS MODELOS: Mismo formato
    # Shape: (batch, time, features) = (1, L, 10)
    # ========================================

    audio_t = torch.tensor(audio, dtype=torch.float32).reshape(1, L, 1)
    knobs_t = torch.tensor(knobs_norm, dtype=torch.float32).reshape(1, 1, 5).repeat(1, L, 1)
    topo_t = torch.tensor(topo_onehot, dtype=torch.float32).reshape(1, 1, 4).repeat(1, L, 1)

    tensor = torch.cat([audio_t, knobs_t, topo_t], dim=2)

    return tensor.to(device)


def run_inference_FINAL(model, model_type, audio, topology, knobs, device='cpu'):
    """
    VERSIÓN FINAL: Todos los modelos reciben el mismo input
    """
    model.eval()

    # Preparar input (mismo formato para todos)
    input_tensor = prepare_input_tensor_FINAL(
        audio, topology, knobs, KMIN, KMAX, model_type, device
    )

    # Inferencia
    with torch.no_grad():
        output = model(input_tensor)

        # Extraer señal
        if output.dim() == 3:
            pred_signal = output.squeeze().cpu().numpy()
        elif output.dim() == 2:
            pred_signal = output.squeeze().cpu().numpy()
        else:
            pred_signal = output.cpu().numpy().flatten()

    return pred_signal


def benchmark_new_config_FINAL(
    models_dict,
    topology,
    knobs,
    signal_type='chirp',
    device='cpu',
    plot=True
):
    """
    VERSIÓN FINAL: Benchmark con formato uniforme
    """
    print("\n" + "="*70)
    print(f"🧪 BENCHMARK: {topology}")
    print(f"   Knobs: {knobs}")
    print(f"   Signal: {signal_type}")
    print("="*70)

    # Generar señal
    input_signal = generate_test_signal(signal_type, FS, DURATION)
    print(f"\n📊 Input signal: shape={input_signal.shape}, range=[{input_signal.min():.3f}, {input_signal.max():.3f}]")

    # Ejecutar inferencia
    results = {}

    for model_name, model in models_dict.items():
        try:
            print(f"\n🔹 {model_name}...")

            pred_signal = run_inference_FINAL(
                model, model_name, input_signal, topology, knobs, device
            )

            # Ajustar longitudes
            L = min(len(input_signal), len(pred_signal))
            pred_signal = pred_signal[:L]
            input_cropped = input_signal[:L]

            results[model_name] = {
                'signal': pred_signal,
                'input': input_cropped
            }

            print(f"   ✅ Output: shape={pred_signal.shape}, range=[{pred_signal.min():.4f}, {pred_signal.max():.4f}]")

        except Exception as e:
            print(f"   ❌ Error: {e}")
            import traceback
            traceback.print_exc()

    # Visualización
    if plot and results:
        plot_results(results, topology, knobs, signal_type)

    print("\n" + "="*70)
    print("✅ Benchmark completado")
    print("="*70)

    return results


print("\n✅ Funciones FINALES cargadas")
print("✅ TODOS los modelos usan input: (1, L, 10)")
print("✅ TCN extrae audio y knobs internamente")



✅ Funciones FINALES cargadas
✅ TODOS los modelos usan input: (1, L, 10)
✅ TCN extrae audio y knobs internamente


# Main Benchmark

In [9]:
class UniversalSpiceEngine:
    def __init__(self, base_path="/content/universal_filter_ngspice/circuits/passive-filters/"):
        self.base_path = base_path

    def run(self, netlist_name, audio_in, params_dict, fs=48000):
        full_path = self.base_path + netlist_name

        # --- A. Inyección de Parámetros (Regex) ---
        try:
            with open(full_path, 'r', encoding='utf-8') as f:
                raw_text = f.read()
        except FileNotFoundError:
            print(f"❌ Error crítico: No encuentro {full_path}")
            return np.zeros_like(audio_in)

        clean_text = re.sub(r"^\.param.*", "*", raw_text, flags=re.IGNORECASE | re.MULTILINE)
        clean_text = re.sub(r"^V.*", "*", clean_text, flags=re.IGNORECASE | re.MULTILINE)

        # Reemplazo dinámico inteligente
        for key, val in params_dict.items():
            pattern = r"\{\s*" + re.escape(key) + r"\s*\}"
            val_fmt = f"{val:.9e}" if 'c' in key.lower() else f"{val:.6f}"
            clean_text = re.sub(pattern, val_fmt, clean_text, flags=re.IGNORECASE)

        # --- B. Simulación ---
        parser = SpiceParser(source=clean_text)
        circuit = parser.build_circuit()

        duration = len(audio_in) / fs
        target_t = np.linspace(0, duration, len(audio_in))

        # Fuente PWL
        pwl = list(zip(target_t, audio_in))
        pwl_str = f'PWL({", ".join([f"{t:.6f} {v:.6f}" for t, v in pwl])})'
        circuit.VoltageSource('AudioSource', 'input', '0', pwl_str)

        simulator = circuit.simulator(temperature=25, nominal_temperature=25)
        # Paso relajado, confíamos en el resampling
        try:
            analysis = simulator.transient(step_time=1.0/fs, end_time=duration)
        except:
            return np.zeros_like(audio_in)

        # --- C. Extracción y Resampling ---
        spice_t = np.array(analysis.time)
        spice_out = None
        for node in analysis.nodes.keys():
            if 'out' in str(node).lower() and 'in' not in str(node).lower():
                spice_out = np.array(analysis.nodes[node])
                break

        if spice_out is None: return np.zeros_like(audio_in)

        # Interpolación para alinear con el modelo
        return np.interp(target_t, spice_t, spice_out)

In [10]:
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq

# ==============================================================================
# 1. CALCULADORA DE MÉTRICAS AVANZADAS (DSP AUDIOPHILE)
# ==============================================================================
def calculate_advanced_metrics(y_true, y_pred, fs, filter_type):
    metrics = {}

    # --- A. Métricas de Tiempo (Time Domain) ---
    # MAE: Error medio absoluto
    metrics['MAE'] = np.mean(np.abs(y_true - y_pred))

    # Pearson Correlation: Similitud de forma (1.0 = idéntico)
    # A veces devuelve NaN si la señal es silencio absoluto, lo manejamos.
    try:
        corr = np.corrcoef(y_true.flatten(), y_pred.flatten())[0, 1]
        metrics['Corr'] = 0 if np.isnan(corr) else corr
    except:
        metrics['Corr'] = 0.0

    # --- B. Análisis Espectral (Frequency Domain) ---
    n = len(y_true)
    xf = fftfreq(n, 1/fs)[:n//2]
    # FFT sin ventana para métricas brutas
    yf_true = np.abs(fft(y_true)[:n//2])
    yf_pred = np.abs(fft(y_pred)[:n//2])

    # Normalización segura
    max_t = np.max(yf_true) + 1e-9
    max_p = np.max(yf_pred) + 1e-9
    mag_true = yf_true / max_t
    mag_pred = yf_pred / max_p

    # --- C. LSD (Log-Spectral Distance) ---
    # Mide la distancia entre las curvas logarítmicas (dB).
    # Es más representativo de cómo escuchamos que el MSE.
    eps = 1e-9
    log_diff = (np.log10(mag_true + eps) - np.log10(mag_pred + eps)) ** 2
    metrics['LSD'] = np.mean(np.sqrt(log_diff))

    # --- D. Detección de Frecuencia de Corte (Tu lógica anterior) ---
    def get_cutoff(mag, freqs, mode):
        threshold = 0.707
        if mode == 'HPF':
            valid_idx = np.where(freqs > 50)[0]
            sub_mag = mag[valid_idx]
            crossings = np.where(sub_mag > threshold)[0]
            if len(crossings) > 0: return freqs[valid_idx[crossings[0]]]
        elif mode == 'LPF':
            valid_idx = np.where(freqs > 100)[0]
            sub_mag = mag[valid_idx]
            crossings = np.where(sub_mag < threshold)[0]
            if len(crossings) > 0: return freqs[valid_idx[crossings[0]]]
        elif mode == 'BPF':
             return freqs[np.argmax(mag)]
        return 0

    def get_notch_center(mag, freqs):
        mask = (freqs > 30) & (freqs < 400)
        if not np.any(mask): return 0
        return freqs[mask][np.argmin(mag[mask])]

    if filter_type == 'Notch':
        fc_true = get_notch_center(mag_true, xf)
        fc_pred = get_notch_center(mag_pred, xf)
    elif filter_type == 'BPF':
        fc_true = get_cutoff(mag_true, xf, 'BPF')
        fc_pred = get_cutoff(mag_pred, xf, 'BPF')
    elif filter_type == 'LPF':
        fc_true = get_cutoff(mag_true, xf, 'LPF')
        fc_pred = get_cutoff(mag_pred, xf, 'LPF')
    else: # HPF
        fc_true = get_cutoff(mag_true, xf, 'HPF')
        fc_pred = get_cutoff(mag_pred, xf, 'HPF')

    metrics['Hz_Diff'] = abs(fc_true - fc_pred)
    metrics['Real_Hz'] = fc_true
    metrics['Pred_Hz'] = fc_pred

    return metrics

In [11]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm  # Barra de progreso para no desesperar

# ==============================================================================
# 1. GENERADOR DE CONFIGURACIONES ALEATORIAS (Validado para Audio)
# ==============================================================================
def generate_random_config(topo_name):
    """
    Genera valores R y C aleatorios pero COHERENTES para audio.
    Evita generar filtros con corte en 0.001Hz o 1MHz.
    """
    # Rangos seguros logarítmicos
    def rand_log(min_val, max_val):
        return 10 ** np.random.uniform(np.log10(min_val), np.log10(max_val))

    config = {}

    # Valores base
    R_val = rand_log(2000, 100000)   # 2k - 100k
    C_val = rand_log(5e-9, 200e-9)   # 5n - 200n

    if topo_name == 'Low-Pass' or topo_name == 'High-Pass':
        # Parametros físicos para Spice
        # Usamos los nombres detectados por el escáner
        if topo_name == 'Low-Pass':
            config['spice_params'] = {"R_gain": R_val, "C_cut": C_val}
        else:
            config['spice_params'] = {"R_gain": R_val, "C_hp": C_val}

        # Parametros para el Modelo
        config['model_knobs'] = {
            'R': f"{R_val:.2f}",
            'C': f"{C_val*1e9:.2f}n"
        }

    elif topo_name == 'Bandpass':
        # Generamos dos etapas distintas
        R1 = rand_log(2000, 50000)
        C1 = rand_log(20e-9, 100e-9) # HPF Stage (bajos)
        R2 = rand_log(2000, 50000)
        C2 = rand_log(1e-9, 10e-9)   # LPF Stage (altos)

        config['spice_params'] = {
            "R_hp": R1, "C_hp": C1,
            "R_lp": R2, "C_lp": C2
        }
        config['model_knobs'] = {
            'R': f"{R1:.2f}", 'C': f"{C1*1e9:.2f}n",
            'R2': f"{R2:.2f}", 'C2': f"{C2*1e9:.2f}n"
        }

    elif topo_name == 'Notch':
        # Twin-T requiere proporciones exactas para funcionar bien
        # R_shunt = R_series / 2
        # C_shunt = C_series * 2
        R_s = rand_log(5000, 50000)
        C_s = rand_log(10e-9, 100e-9)

        config['spice_params'] = {
            "R_series": R_s, "C_series": C_s,
            "R_shunt": R_s / 2, "C_shunt": C_s * 2
        }
        # El modelo aprendió que R2 es R/2 y C2 es C*2, pero se lo pasamos explícito
        config['model_knobs'] = {
            'R': f"{R_s:.2f}", 'C': f"{C_s*1e9:.2f}n",
            'R2': f"{R_s/2:.2f}", 'C2': f"{C_s*2*1e9:.2f}n"
        }

    return config

# ==============================================================================
# 2. MOTOR ESTADÍSTICO (Monte Carlo)
# ==============================================================================
def run_statistical_benchmark(model, model_name, n_iter=30):
    print(f"🚀 Iniciando Benchmark Estadístico para {model_name} ({n_iter} it/filtro)...")

    topologies = [
        {"name": "Low-Pass", "netlist": "low_pass_filter.cir", "type": "LPF"},
        {"name": "High-Pass", "netlist": "high_pass_filter.cir", "type": "HPF"},
        {"name": "Bandpass", "netlist": "band_pass_filter.cir", "type": "BPF"},
        {"name": "Notch", "netlist": "band_stop_filter.cir", "type": "Notch"},
    ]

    engine = UniversalSpiceEngine()

    # Estructura para guardar resultados
    stats_report = []

    for topo in topologies:
        metrics_accum = {'MAE': [], 'LSD': [], 'Corr': [], 'Hz_Diff': []}

        # Barra de progreso por topología
        pbar = tqdm(range(n_iter), desc=f"Testing {topo['name']}", leave=True)

        valid_runs = 0

        for i in pbar:
            # 1. Generar configuración aleatoria
            config = generate_random_config(topo['name'])

            # Generar señal nueva cada vez (para variar el ruido/fase)
            # Usamos Chirp porque es lo mejor para medir Hz
            audio_in = generate_test_signal('chirp', FS, 0.5)

            try:
                # 2. Simulación SPICE
                y_spice = engine.run(topo['netlist'], audio_in, config['spice_params'], FS)

                # Chequeo de seguridad
                if np.max(np.abs(y_spice)) < 1e-6: continue

                # 3. Predicción Modelo
                y_pred = run_inference_FINAL(model, model_name, audio_in,
                                            topo['name'], config['model_knobs'], DEVICE)

                # Recorte y Métricas
                L = min(len(y_spice), len(y_pred))
                m = calculate_advanced_metrics(y_spice[:L], y_pred[:L], FS, topo['type'])

                # Guardar métricas
                metrics_accum['MAE'].append(m['MAE'])
                metrics_accum['LSD'].append(m['LSD'])
                metrics_accum['Corr'].append(m['Corr'])
                metrics_accum['Hz_Diff'].append(m['Hz_Diff'])

                valid_runs += 1

            except Exception as e:
                pass # Ignoramos fallos puntuales de simulación para no detener el loop

        # --- CÁLCULO ESTADÍSTICO FINAL POR TOPOLOGÍA ---
        if valid_runs > 0:
            row = {
                "Topology": topo['name'],
                "N": valid_runs,
                "MAE": f"{np.mean(metrics_accum['MAE']):.4f} ± {np.std(metrics_accum['MAE']):.4f}",
                "LSD (dB)": f"{np.mean(metrics_accum['LSD']):.3f} ± {np.std(metrics_accum['LSD']):.3f}",
                "Hz Diff": f"{np.mean(metrics_accum['Hz_Diff']):.1f} ± {np.std(metrics_accum['Hz_Diff']):.1f}",
                "Corr": f"{np.mean(metrics_accum['Corr']):.4f} ± {np.std(metrics_accum['Corr']):.4f}"
            }
            stats_report.append(row)

    return pd.DataFrame(stats_report)

2026-02-06 20:05:26,636 - numexpr.utils._init_num_threads - INFO - NumExpr defaulting to 2 threads.


In [12]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

# ==============================================================================
# CONFIGURACIÓN
# ==============================================================================
DURATION = 0.1
CURRENT_FS = 48000

# ==============================================================================
# 1. MOTOR ESTADÍSTICO (Actualizado con duración corta)
# ==============================================================================
def run_statistical_benchmark(model, model_name, n_iter=10):
    print(f"🚀 Benchmark ({n_iter} it/filtro) | Duración audio: {DURATION}s")

    topologies = [
        {"name": "Low-Pass", "netlist": "low_pass_filter.cir", "type": "LPF"},
        {"name": "High-Pass", "netlist": "high_pass_filter.cir", "type": "HPF"},
        {"name": "Bandpass", "netlist": "band_pass_filter.cir", "type": "BPF"},
        {"name": "Notch", "netlist": "band_stop_filter.cir", "type": "Notch"},
    ]

    engine = UniversalSpiceEngine()
    stats_report = []

    for topo in topologies:
        metrics_accum = {'MAE': [], 'LSD': [], 'Corr': [], 'Hz_Diff': []}

        # Barra de progreso limpia
        pbar = tqdm(range(n_iter), desc=f"{model_name[:4]}->{topo['name']}", leave=False)

        valid_runs = 0

        for i in pbar:
            config = generate_random_config(topo['name'])

            audio_in = generate_test_signal('chirp', CURRENT_FS, DURATION)

            try:
                # 1. Simulación SPICE
                y_spice = engine.run(topo['netlist'], audio_in, config['spice_params'], CURRENT_FS)
                if np.max(np.abs(y_spice)) < 1e-6: continue

                # 2. Predicción Modelo
                y_pred = run_inference_FINAL(model, model_name, audio_in,
                                            topo['name'], config['model_knobs'], DEVICE)

                # 3. Métricas
                L = min(len(y_spice), len(y_pred))
                m = calculate_advanced_metrics(y_spice[:L], y_pred[:L], CURRENT_FS, topo['type'])

                metrics_accum['MAE'].append(m['MAE'])
                metrics_accum['LSD'].append(m['LSD'])
                metrics_accum['Corr'].append(m['Corr'])
                metrics_accum['Hz_Diff'].append(m['Hz_Diff'])
                valid_runs += 1

            except Exception:
                pass

        # Promedios
        if valid_runs > 0:
            row = {
                "Topology": topo['name'],
                "N": valid_runs,
                # Formato string "Media ± Desv"
                "MAE": f"{np.mean(metrics_accum['MAE']):.4f} ± {np.std(metrics_accum['MAE']):.4f}",
                "LSD (dB)": f"{np.mean(metrics_accum['LSD']):.3f} ± {np.std(metrics_accum['LSD']):.3f}",
                "Hz Diff": f"{np.mean(metrics_accum['Hz_Diff']):.1f} ± {np.std(metrics_accum['Hz_Diff']):.1f}",
                "Corr": f"{np.mean(metrics_accum['Corr']):.4f} ± {np.std(metrics_accum['Corr']):.4f}"
            }
            stats_report.append(row)

    return pd.DataFrame(stats_report)

# ==============================================================================
# 2. EJECUCIÓN BATTLE ROYALE 🥊
# ==============================================================================

# Si quieres ir rápido para probar que funciona, deja N_ITER bajo (ej: 5)
# Si quieres el paper final, ponle 30.
N_ITER = 5
MODELS_TO_TEST = list(models.keys())

print(f"🥊 INICIANDO BATTLE ROYALE (Modo Turbo)")
all_results_list = []

for model_name in MODELS_TO_TEST:
    curr_model = models[model_name]
    try:
        df_model = run_statistical_benchmark(curr_model, model_name, n_iter=N_ITER)
        df_model.insert(0, 'Model', model_name)
        all_results_list.append(df_model)
        print(f"✅ {model_name} Ok.")
    except Exception as e:
        print(f"❌ Error en {model_name}: {e}")

# ==============================================================================
# 3. VISUALIZACIÓN SEGURA (Sin KeyErrors)
# ==============================================================================
if all_results_list:
    final_df = pd.concat(all_results_list, ignore_index=True)

    # --- CORRECCIÓN IMPORTANTE ---
    # No ordenamos por MAE porque es texto ("0.05 ± 0.01").
    # Ordenamos por Modelo y Topología para agrupar visualmente.
    if 'Topology' in final_df.columns:
        final_df = final_df.sort_values(by=['Model', 'Topology'])

    print("\n🏆 TABLA MAESTRA DE RESULTADOS 🏆")
    pd.set_option('display.max_rows', None)
    display(final_df)

    # Guardar
    final_df.to_csv('final_benchmark_fast.csv', index=False)
else:
    print("⚠️ No data.")

🥊 INICIANDO BATTLE ROYALE (Modo Turbo)
🚀 Benchmark (5 it/filtro) | Duración audio: 0.1s


LSTM->Low-Pass:   0%|          | 0/5 [00:00<?, ?it/s]

LSTM->High-Pass:   0%|          | 0/5 [00:00<?, ?it/s]

LSTM->Bandpass:   0%|          | 0/5 [00:00<?, ?it/s]

LSTM->Notch:   0%|          | 0/5 [00:00<?, ?it/s]

...

RNN->High-Pass:   0%|          | 0/5 [00:00<?, ?it/s]

RNN->Bandpass:   0%|          | 0/5 [00:00<?, ?it/s]

RNN->Notch:   0%|          | 0/5 [00:00<?, ?it/s]

✅ RNN Ok.

🏆 TABLA MAESTRA DE RESULTADOS 🏆


,Model,Topology,N,MAE,LSD (dB),Hz Diff,Corr
2,LSTM,Bandpass,5,0.0269 ± 0.0252,0.109 ± 0.062,44.0 ± 45.9,0.9900 ± 0.0141
1,LSTM,High-Pass,5,0.0537 ± 0.0260,0.139 ± 0.053,36.0 ± 45.9,0.9841 ± 0.0115
0,LSTM,Low-Pass,5,0.0321 ± 0.0385,0.069 ± 0.035,0.0 ± 0.0,0.9804 ± 0.0368
3,LSTM,Notch,5,0.1231 ± 0.0157,0.487 ± 0.107,24.0 ± 22.4,0.9074 ± 0.0259
10,RNN,Bandpass,5,0.1043 ± 0.0246,0.460 ± 0.107,216.0 ± 198.3,0.8645 ± 0.0695
9,RNN,High-Pass,5,0.1467 ± 0.0244,0.229 ± 0.081,120.0 ± 93.2,0.9237 ± 0.0268
8,RNN,Low-Pass,5,0.0891 ± 0.0381,0.304 ± 0.067,0.0 ± 0.0,0.9347 ± 0.0297
11,RNN,Notch,5,0.1696 ± 0.0098,0.299 ± 0.015,60.0 ± 41.0,0.8764 ± 0.0120
6,TCN,Bandpass,5,0.0683 ± 0.0361,0.471 ± 0.151,514.0 ± 155.0,0.9310 ± 0.0550
5,TCN,High-Pass,5,0.0639 ± 0.0275,0.253 ± 0.185,322.0 ± 514.0,0.9750 ± 0.0136


In [13]:
# import re
# import numpy as np
# import matplotlib.pyplot as plt
# from PySpice.Spice.Parser import SpiceParser

# # ============================================================
# # 1. GENERADOR DE GROUND TRUTH (ALINEADO)
# # ============================================================
# def run_spice_aligned(audio_in, r_val, c_val, fs=48000):
#     path = "/content/universal_filter_ngspice/circuits/passive-filters/low_pass_filter.cir"

#     # --- A. Lectura y Limpieza ---
#     try:
#         with open(path, 'r', encoding='utf-8') as f:
#             raw_text = f.read()
#     except FileNotFoundError:
#         return np.zeros_like(audio_in)

#     # Borrar params viejos y fuentes viejas
#     clean_text = re.sub(r"^\.param.*", "*", raw_text, flags=re.IGNORECASE | re.MULTILINE)
#     clean_text = re.sub(r"^V.*", "*", clean_text, flags=re.IGNORECASE | re.MULTILINE)

#     # Inyectar valores numéricos directos (Hardcoding por Regex)
#     clean_text = re.sub(r"\{\s*R_gain\s*\}", f"{r_val:.6f}", clean_text, flags=re.IGNORECASE)
#     clean_text = re.sub(r"\{\s*C_cut\s*\}", f"{c_val:.9e}", clean_text, flags=re.IGNORECASE)

#     # --- B. Configuración de Simulación ---
#     parser = SpiceParser(source=clean_text)
#     circuit = parser.build_circuit()

#     # Definir eje temporal objetivo (el del audio/modelo)
#     duration = len(audio_in) / fs
#     target_time_axis = np.linspace(0, duration, len(audio_in))

#     # Fuente PWL
#     pwl_pairs = list(zip(target_time_axis, audio_in))
#     pwl_string = f'PWL({", ".join([f"{t:.6f} {v:.6f}" for t, v in pwl_pairs])})'
#     circuit.VoltageSource('AudioSource', 'input', '0', pwl_string)

#     # --- C. Simular ---
#     simulator = circuit.simulator(temperature=25, nominal_temperature=25)
#     try:
#         # NGSpice decidirá sus propios pasos de tiempo internamente
#         analysis = simulator.transient(step_time=1.0/fs, end_time=duration)
#     except Exception as e:
#         print(f"💥 Crash SPICE: {e}")
#         return np.zeros_like(audio_in)

#     # --- D. Extraer y Resamplear ---
#     spice_time = np.array(analysis.time)
#     spice_output = None

#     # Buscar nodo de salida
#     for node in analysis.nodes.keys():
#         n_str = str(node).lower()
#         if ('output' in n_str or 'out' in n_str) and 'input' not in n_str:
#             spice_output = np.array(analysis.nodes[node])
#             break

#     if spice_output is None:
#         return np.zeros_like(audio_in)

#     # INTERPOLACIÓN MÁGICA: Ajustar SPICE a 48kHz fijo
#     y_aligned = np.interp(target_time_axis, spice_time, spice_output)

#     return y_aligned

# # ============================================================
# # 2. PLOTEO CON ZOOM
# # ============================================================
# def plot_suite_zoom(title, y_spice, model_results, fc_theory):
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

#     # --- PLOT 1: PANORAMA COMPLETO ---
#     x_axis = np.arange(len(y_spice))
#     ax1.plot(x_axis, y_spice, color='black', linewidth=2, alpha=0.5, label='GT (Spice)')

#     colors = ['#ff7f0e', '#2ca02c', '#d62728']
#     if model_results:
#         for i, (name, data) in enumerate(model_results.items()):
#             y_pred = data['signal']
#             L = min(len(y_pred), len(y_spice))
#             ax1.plot(x_axis[:L], y_pred[:L], color=colors[i], linestyle='--', linewidth=1, label=name)

#     ax1.set_title(f"{title} (fc={fc_theory:.0f}Hz) - VISTA COMPLETA")
#     ax1.set_xlabel("Muestras")
#     ax1.set_ylabel("V")
#     ax1.legend()
#     ax1.grid(True, alpha=0.3)

#     # --- PLOT 2: ZOOM (Muestras 2000 a 2400 - Inicio del Chirp) ---
#     # Elegimos una ventana donde haya señal visible
#     start, end = 2000, 2400
#     if len(y_spice) > end:
#         ax2.plot(x_axis[start:end], y_spice[start:end], color='black', linewidth=3, alpha=0.5, label='GT')

#         if model_results:
#             for i, (name, data) in enumerate(model_results.items()):
#                 y_pred = data['signal']
#                 mae_local = np.mean(np.abs(y_pred[start:end] - y_spice[start:end]))
#                 ax2.plot(x_axis[start:end], y_pred[start:end], color=colors[i], linestyle='--', linewidth=2, label=f"{name} (Err: {mae_local:.3f})")

#         ax2.set_title("🔍 ZOOM (Detalle de Fase/Amplitud)")
#         ax2.set_xlabel("Muestras")
#         ax2.legend()
#         ax2.grid(True, alpha=0.3)

#     plt.tight_layout()
#     plt.show()

# # ============================================================
# # 3. EJECUCIÓN
# # ============================================================

# print("\n" + "🎧"*40)
# print("SUITE FINAL ALINEADA + ZOOM")
# print("🎧"*40)

# # --- CONFIGURACIONES ---
# tests = [
#     ("Low-Pass 100Hz",  {'R': '10k', 'C': '150n'}, 10000, 150e-9),
#     ("Low-Pass 1kHz",   {'R': '3.3k', 'C': '47n'}, 3300,  47e-9),
#     ("Low-Pass 5kHz",   {'R': '2.2k', 'C': '15n'}, 2200,  15e-9)
# ]

# for name, knobs, R_val, C_val in tests:
#     fc = 1 / (2 * np.pi * R_val * C_val)
#     print(f"\n🔹 PROCESANDO: {name} (fc ≈ {fc:.1f} Hz)...")

#     try:
#         # 1. Modelos (Predicción)
#         # Nota: Usamos 'chirp' porque es lo mejor para ver respuesta en frecuencia
#         results = benchmark_new_config_FINAL(models, 'Low-Pass', knobs, 'chirp', DEVICE, plot=False)
#         audio_in = results[list(results.keys())[0]]['audio_in']

#         # 2. Spice (Ground Truth Alineado)
#         y_spice = run_spice_aligned(audio_in, R_val, C_val)

#         # 3. Verificación Rápida de Energía
#         peak_spice = np.max(np.abs(y_spice))
#         print(f"   📊 Spice Peak: {peak_spice:.4f} V")

#         if peak_spice < 0.001:
#             print("   ⚠️ WARNING: Señal plana en Spice. ¿Revisar circuito?")

#         # 4. Gráfica
#         plot_suite_zoom(name, y_spice, results, fc)

#     except Exception as e:
#         print(f"   ❌ ERROR en {name}: {e}")

# print("\n✅ Suite terminada.")